In [32]:
import numpy as np
import pandas as pd
from itertools import product
import unicodedata


In [34]:
# Función para remover acentos y estandarizar nombres
def normalizar_nombre(nombre):
    return ''.join(c for c in unicodedata.normalize('NFKD', nombre) if not unicodedata.combining(c))

In [36]:
# Cargar datos
posiciones_path = "C:/temp/Projects/Tabla_posiciones.csv"
partidos_path = "C:/temp/Projects/PartidosFaltantes.csv"

In [38]:
tabla_posiciones = pd.read_csv(posiciones_path, encoding="latin1")
partidos_faltantes = pd.read_csv(partidos_path, encoding="latin1")

In [40]:
# Normalizar nombres de equipos
tabla_posiciones["Country"] = tabla_posiciones["Country"].apply(normalizar_nombre)
partidos_faltantes["Local"] = partidos_faltantes["Local"].apply(normalizar_nombre)
partidos_faltantes["Visitor"] = partidos_faltantes["Visitor"].apply(normalizar_nombre)

In [ ]:
# Actualizar puntos por el empate 0-0 entre Bolivia y Uruguay
#tabla_posiciones.loc[tabla_posiciones["Country"] == "Bolivia", "PTS"] += 1
#tabla_posiciones.loc[tabla_posiciones["Country"] == "Uruguay", "PTS"] += 1

In [ ]:
# Eliminar el partido ya jugado de los partidos faltantes
#partidos_faltantes = partidos_faltantes[~((partidos_faltantes["Local"] == "Bolivia") & (partidos_faltantes["Visitor"] == "Uruguay"))]


In [42]:
# Extraer equipos y puntos actuales
equipos = tabla_posiciones["Country"].tolist()
puntos_actuales = dict(zip(tabla_posiciones["Country"], tabla_posiciones["PTS"]))

In [ ]:
# ---------- SIMULACIÓN DE TODAS LAS COMBINACIONES ----------

goles_posibles = range(4)  # 0 a 4 goles
partidos_restantes = list(zip(partidos_faltantes["Local"], partidos_faltantes["Visitor"]))
num_partidos = len(partidos_restantes)

In [ ]:
# Generar combinaciones de goles para todos los partidos
combinaciones_goles = product(goles_posibles, repeat=2 * num_partidos)


In [ ]:
# Contadores
position_counts = {team: np.zeros(len(equipos)) for team in equipos}
classification_counts = {team: {"Clasificado": 0, "Repechaje": 0, "Eliminado": 0} for team in equipos}

total_combinaciones = 0

for goles_comb in combinaciones_goles:
    total_combinaciones += 1
    puntos = puntos_actuales.copy()
    dif_goles = {team: 0 for team in equipos}

    for i, (local, visitante) in enumerate(partidos_restantes):
        gl = goles_comb[2 * i]
        gv = goles_comb[2 * i + 1]

        dif_goles[local] += gl - gv
        dif_goles[visitante] += gv - gl

        if gl > gv:
            puntos[local] += 3
        elif gl < gv:
            puntos[visitante] += 3
        else:
            puntos[local] += 1
            puntos[visitante] += 1

    ordenados = sorted(puntos.items(), key=lambda x: (x[1], dif_goles[x[0]]), reverse=True)

    for pos, (team, _) in enumerate(ordenados):
        position_counts[team][pos] += 1
        if pos < 6:
            classification_counts[team]["Clasificado"] += 1
        elif pos == 6:
            classification_counts[team]["Repechaje"] += 1
        else:
            classification_counts[team]["Eliminado"] += 1


In [ ]:
# ---------- RESULTADOS ----------

# Probabilidades de posición
df_posiciones = pd.DataFrame({
    team: counts / total_combinaciones * 100
    for team, counts in position_counts.items()
}, index=[f"Pos {i+1}" for i in range(len(equipos))]).T.round(2)

# Probabilidades de clasificación
df_clasificacion = pd.DataFrame({
    team: {
        k: v / total_combinaciones * 100
        for k, v in counts.items()
    } for team, counts in classification_counts.items()
}).T.round(2)


In [ ]:
# Mostrar
print("\n=== Probabilidades por Posición (%) ===")
print(df_posiciones)
print("\n=== Probabilidades de Clasificación (%) ===")
print(df_clasificacion)

In [ ]:
import matplotlib.pyplot as plt

# ----- GRÁFICO 1: Probabilidades por posición (barras apiladas) -----
df_posiciones_sorted = df_posiciones.loc[df_posiciones.sum(axis=1).sort_values(ascending=True).index]

ax = df_posiciones_sorted.plot(kind='bar', stacked=True, figsize=(14, 6), colormap='tab10')
plt.title("Probabilidades por Posición (%)")
plt.ylabel("Porcentaje")
plt.xlabel("Equipo")
plt.legend(title="Posición", bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.grid(True, axis='y')
plt.show()

# ----- GRÁFICO 2: Clasificación (barras agrupadas) -----
df_clasificacion_sorted = df_clasificacion.loc[df_posiciones_sorted.index]

df_clasificacion_sorted.plot(kind="bar", figsize=(14, 6), colormap="Set2")
plt.title("Probabilidades de Clasificación (%)")
plt.ylabel("Porcentaje")
plt.xlabel("Equipo")
plt.xticks(rotation=45)
plt.grid(True, axis='y')
plt.tight_layout()
plt.show()